In [ ]:
import numpy as np
import pandas as pd


In [ ]:
from google.colab import files
path_to_file=list(files.upload().keys())[0]

In [ ]:
with open(path_to_file,'r', encoding='utf-8') as file:
  text=file.read()



In [ ]:
import tensorflow as tf

In [ ]:


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential


In [ ]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
total_words=len(tokenizer.word_index)+1



In the above code, the text is tokenized, which means it is divided into individual words or tokens. The Tokenizer' object is created, which will handle the tokenization process. The 'fit_on_texts' method of the tokenizer is called, passing the 'text' as input. This method analyzes the text and builds a vocabulary of unique words, assigning each word a numerical index. The 'total_words' variable is then assigned the value of the length of the word index plus one, representing the total number of distinct words in the text.In the above code, the text is tokenized, which means it is divided into individual words or tokens. The Tokenizer' object is created, which will handle the tokenization process. The 'fit_on_texts' method of the tokenizer is called, passing the 'text' as input. This method analyzes the text and builds a vocabulary of unique words, assigning each word a numerical index. The 'total_words' variable is then assigned the value of the length of the word index plus one, representing the total number of distinct words in the text.

now lets create input - output pairs by splitting the text into sequence of tokens and forming  n-grams form the sequence

In [ ]:
input_sequences=[]
for line in text.split('\n'):
  token_list=tokenizer.texts_to_sequences([line])[0]
  for i in range(1,len(token_list)):
    n_gram_sequence=token_list[:i+1]
    input_sequences.append(n_gram_sequence)

In [ ]:
max_sequence_len=max([len(seq) for seq in input_sequences])
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre'))

In the above code, the input sequences are padded to ensure all sequences have the same length.
The variable 'max_sequence_len' is assigned the maximum length among all the input sequences.
The 'pad_sequences' function is used to pad or truncate the input sequences to match this maximum length.
The 'pad_sequences' function takes the input_sequences list, sets the maximum length to 'max sequence_len',
and specifies that the padding should be added at the beginning of each sequence using the 'padding=pre' argument. Finally,
the input sequences are converted into a numpy array to facilitate further processing.
 Now let's split the sequences into input and output:

In [ ]:
x=input_sequences[:,:-1]
y=input_sequences[:,:-1]

In [ ]:
 y= np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

 In the above code, the input sequences are split into two arrays, 'X' and 'y', to create the input and output for training the next word prediction model. The 'X' array is assigned the values of all rows in the 'input_sequences' array except for the last column. It means that 'X' contains all the tokens in each sequence except for the last one, representing the input context.

In [ ]:

from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
model = Sequential()
model.add(Embedding (total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

 The code above defines the model architecture for the next word prediction model. The 'Sequential' model is created, which represents a linear stack of layers.
 The first layer added to the model is the 'Embedding' layer, which is responsible for converting the input sequences into dense vectors of fixed size. It takes three arguments:
 (1). 'total_words', which represents the total number of distinct words in the vocabulary;
 (2). '100', which denotes the dimensionality of the word embeddings;
 (3). and 'input length', which specifies the length of the input sequences.
 The next layer added is the 'LSTM' layer, a type of recurrent neural network (RNN) layer designed for capturing sequential dependencies in the data.
 It has 150 units, which means it will learn 150 internal representations or memory cells.

 Finally, the 'Dense' layer is added, which is a fully connected layer that produces the output predictions.
 It has 'total_words' units and uses the 'softmax' activation function to convert the predicted scores into probabilities, indicating
 the likelihood of each word being the next one in the sequence.
 Now let's compile and train the model:

In [ ]:
model. compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)



In the above code, the model is being compiled and trained. The 'compile' method configures the model for training. The 'loss' parameter is set
to 'categorical_crossentropy', a commonly used loss function for multi-class classification problems. The 'optimizer' parameter is set to 'adam',
an optimization algorithm that adapts the learning rate during training.

The 'metrics' parameter is set to 'accuracy' to monitor the accuracy during training. After compiling the model, the 'fit' method is called to train
the model on the input sequences 'X' and the corresponding output 'y'. The 'epochs' parameter specifies the number of times the training
process will iterate over the entire dataset. The 'verbose' parameter is set to '1' to display the training process.
The above code will take more than an hour to execute. Once the code is executed, here's how we can generate the next word predictions using
our model:

In [ ]:
# seed_text = "I will leave if they"
seed_text = "Are you"
next_words = 3

for _ in range(next_words):
token_list = tokenizer.texts_to_sequences([seed_text])[ø]
token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
predicted = np.argmax(model.predict(token_list), axis =- 1)
output_word = ""
for word, index in tokenizer.word_index.items():
if index == predicted:
output_word = word
break
seed_text += " " + output_word

print(seed_text)